<a href="https://colab.research.google.com/github/mohammedshaneeb-ai/Get-Started_with_Pytorch/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [18]:
# Download training data
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

In [19]:
# Creating test data
test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform=ToTensor()
)

In [20]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)


In [21]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
    )

In [22]:
print(f"using {device} device")

using cuda device


In [23]:
# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits


model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [24]:
# Optimizing the model Parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In [25]:
def train(dataloader,model,loss_fn,optimizer):
  size = len(dataloader.dataset)
  model.train()
  for  batch,(X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # compute prediction error
    pred = model(X)
    loss = loss_fn(pred,y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


    if batch % 100 == 0:
      loss,current = loss.item(),(batch +1) * len(X)
      print(f"loss: {loss :>7f}[{current:>5d}/{size:>5d}]")


In [26]:
print(len(train_dataloader.dataset))
print(len(train_dataloader))

60000
938


In [27]:
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [28]:
epochs = 5
for t in range(epochs):
  print(f"Epoch{t+1}\n-----------------")
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader,model,loss_fn)
print("Done!")

Epoch1
-----------------
loss: 2.307416[   64/60000]
loss: 2.285498[ 6464/60000]
loss: 2.266133[12864/60000]
loss: 2.256382[19264/60000]
loss: 2.225286[25664/60000]
loss: 2.212332[32064/60000]
loss: 2.212331[38464/60000]
loss: 2.174802[44864/60000]
loss: 2.175035[51264/60000]
loss: 2.135781[57664/60000]
Test Error: 
 Accuracy: 47.0%, Avg loss: 2.130928 

Epoch2
-----------------
loss: 2.143089[   64/60000]
loss: 2.126419[ 6464/60000]
loss: 2.063003[12864/60000]
loss: 2.082289[19264/60000]
loss: 2.010707[25664/60000]
loss: 1.965438[32064/60000]
loss: 1.984759[38464/60000]
loss: 1.895875[44864/60000]
loss: 1.907583[51264/60000]
loss: 1.830725[57664/60000]
Test Error: 
 Accuracy: 56.6%, Avg loss: 1.829182 

Epoch3
-----------------
loss: 1.862166[   64/60000]
loss: 1.831496[ 6464/60000]
loss: 1.708574[12864/60000]
loss: 1.757531[19264/60000]
loss: 1.631796[25664/60000]
loss: 1.603658[32064/60000]
loss: 1.618495[38464/60000]
loss: 1.520115[44864/60000]
loss: 1.552771[51264/60000]
loss: 1.4